In [9]:
from keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import os
import shutil

In [10]:
# Define the data generator with the desired data augmentation
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [12]:
def augment_data(data_dir, aug_dir, merged_dir, n_augmented = 5):
    # Get the list of tasks in the data directory
    tasks = os.listdir(data_dir)

    # Loop over each task
    for task in tasks:
        task_dir = os.path.join(data_dir, task)

        # Get the list of classes in the task directory
        classes = os.listdir(task_dir)

        # Loop over each class
        for class_name in classes:
            class_dir = os.path.join(task_dir, class_name)

            # Create a directory to save the augmented images
            save_dir = os.path.join(aug_dir, task, class_name)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            # Create a data generator for the class
            generator = datagen.flow_from_directory(
                task_dir,
                classes=[class_name],
                target_size=(224, 224),
                batch_size=1,
                class_mode='categorical',
                save_to_dir=save_dir,
                save_prefix='aug',
                save_format='jpg'
            )

            # Generate augmented images for each image in the class
            n_original = len(os.listdir(class_dir))
            for i in range(n_augmented * n_original):
                next(generator)

            # Print the total number of images for this class
            n_total = n_original + n_augmented * n_original
            print(f'Total number of images for {task}/{class_name}: {n_total}')

    # Move augmented images to the merged directory
    for root, _, files in os.walk(aug_dir):
        for file in files:
            src_file = os.path.join(root, file)
            dest_file = os.path.join(merged_dir, os.path.relpath(src_file, aug_dir))
            os.makedirs(os.path.dirname(dest_file), exist_ok=True)
            shutil.move(src_file, dest_file)

    # Copy original images to the merged directory
    for root, _, files in os.walk(data_dir):
        for file in files:
            src_file = os.path.join(root, file)
            dest_file = os.path.join(merged_dir, os.path.relpath(src_file, data_dir))
            os.makedirs(os.path.dirname(dest_file), exist_ok=True)
            shutil.copy(src_file, dest_file)

    # Optionally, delete the data_dir and aug_dir if needed
    shutil.rmtree(aug_dir)

In [13]:
augment_data("D:/msc/data/train", "D:/msc/data/temp", "D:/msc/data/train_augumented")

Found 1626 images belonging to 1 classes.
Total number of images for task1/combined: 9756
Found 374 images belonging to 1 classes.
Total number of images for task1/mel: 2244
Found 1746 images belonging to 1 classes.
Total number of images for task2/combined: 10476
Found 254 images belonging to 1 classes.
Total number of images for task2/Seb: 1524


In [15]:
augment_data("D:/msc/data/train_merged", "D:/msc/data/temp", "D:/msc/data/train_merged_augumented")

Found 1746 images belonging to 1 classes.
Total number of images for task1/combined: 10476
Found 404 images belonging to 1 classes.
Total number of images for task1/mel: 2424
Found 1854 images belonging to 1 classes.
Total number of images for task2/combined: 11124
Found 296 images belonging to 1 classes.
Total number of images for task2/seb: 1776
